In [ ]:
from imageai.Detection import ObjectDetection
import os
from imageai.Detection.Custom import DetectionModelTrainer
from PIL import Image
import cv2
import numpy as np

In [ ]:
file = 'p9.jpg'
execution_path = os.getcwd()
detector = ObjectDetection()
detector.setModelTypeAsTinyYOLOv3()
detector.setModelPath(os.path.join(execution_path , "yolo-tiny.h5"))
detector.loadModel()
im = cv2.imread(os.path.join(execution_path , file))
im = cv2.resize(im, (600,400))
cv2.imwrite(os.path.join(execution_path, 'input.jpg'), im)

In [ ]:
detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , "input.jpg"), output_image_path=os.path.join(execution_path , "new.jpg"), minimum_percentage_probability = 20)

In [ ]:
img_path = os.path.join(execution_path , "input.jpg")
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

In [ ]:
def get_cent_coords(detections, img):
    centres = []
    coords = []
    for i,j in enumerate(detections):
        if j['name'] == 'person':
            x1 = j['box_points'][0]
            y1 = j['box_points'][1]
            x2 = j['box_points'][2]
            y2 = j['box_points'][3]
            centre = (int((x2+x1)/2), int((y2+y1)/2))
            centres.append(centre)
            coords.append([x1, y1, x2, y2])
    return coords, centres

In [ ]:
coords, centres = get_cent_coords(detections, img)

In [ ]:
def find_people(centres, coords, img):
    yes = []
    no = []

    for i,j in enumerate(centres):
        for k,l in enumerate(centres):
            if k != i:
                dist = abs(centres[k][0] - centres[i][0])
                if dist < 60 and dist > 20:
                    x1 = coords[i][0]
                    x2 = coords[i][2]
                    y1 = coords[i][1]
                    y2 = coords[i][3]
                    if k not in no:
                        no.append(k)
                else:
                    x1 = coords[i][0]
                    x2 = coords[i][2]
                    y1 = coords[i][1]
                    y2 = coords[i][3]
                    if k not in yes:
                        yes.append(k)
    return yes, no

In [ ]:
yes, no = find_people(centres, coords, img)

In [ ]:
yes = list(set(yes))
no = list(set(no))
for n in no:
    if n in yes:
        yes.remove(n)

In [ ]:
def mark_people(yes, no, coords, img):    
    for y in yes:
        x1 = coords[y][0]
        y1 = coords[y][1]
        x2 = coords[y][2]
        y2 = coords[y][3]
    
        cv2.rectangle(img, (x1, y1) ,(x2, y2), (0, 255, 0), 2)
        cv2.rectangle(img, (x1, y2 - 25), (x2, y2), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(img, "Follow", (x1 + 6, y2 - 6), font, 0.5, (255, 255, 255), 1)
    
    for y in no:
        x1 = coords[y][0]
        y1 = coords[y][1]
        x2 = coords[y][2]
        y2 = coords[y][3]
    
        cv2.rectangle(img, (x1, y1) ,(x2, y2), (255, 0, 0), 2)
        cv2.rectangle(img, (x1, y2 - 25), (x2, y2), (255, 0, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(img, "No_Follow", (x1 + 6, y2 - 6), font, 0.5, (255, 255, 255), 1)
    cv2.imwrite(os.path.join('results',('-').join([file.split('.')[0], 'results.jpg'])), img)
    
    return img

In [ ]:
final_img = mark_people(yes, no, coords, img)
pil = Image.fromarray(final_img)

In [ ]:
pil

## VIDEO DETECTION

In [ ]:
k = 400
def video_detection(model, input_file, output_file, fps=30):
    """
    Returns detections from video and save them in separate files
    """
    # Read in the video
    video = cv2.VideoCapture(input_file)

    # Video frame dimensions
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    frame_width = int(frame_width/2)
    frame_height = int(frame_height/2)
    
    # Scale down frames when passing into model for faster speeds
    scaled_size = 800
    scale_down_factor = min(frame_height, frame_width) / scaled_size

    # The VideoWriter with which we'll write our video with the boxes and labels
    # Parameters: filename, fourcc, fps, frame_size
    out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'DIVX'), fps, (frame_width, frame_height))

    # Loop through every frame of the video
    while True:
        ret, frame = video.read()
        # Stop the loop when we're done with the video
        if not ret:
            break

        resized_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)    
        cv2.imwrite(os.path.join('video/','input.jpg'), resized_frame)
        detections = model.detectObjectsFromImage(input_image=os.path.join('video/','input.jpg'), output_image_path=os.path.join('video/' , "new.jpg"), minimum_percentage_probability = 20)
        img_path = os.path.join('video/' , "input.jpg")
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        coords, centres = get_cent_coords(detections, img)
        yes, no = find_people(centres, coords, img)

        yes = list(set(yes))
        no = list(set(no))
        for n in no:
            if n in yes:
                yes.remove(n)
                
        final_img = mark_people(yes, no, coords, img)
        cv2.imwrite(os.path.join('video/','new2.jpg'), final_img)
        # Write this frame to our video file
        out.write(final_img)

        # If the 'q' key is pressed, break from the loop
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    # When finished, release the video capture and writer objects
    video.release()
    out.release()

    # Close all the frames
    cv2.destroyAllWindows()    

In [ ]:
video_detection(detector, 'video.mp4', 'output/video2.avi', 30)

## LIVE VIDEO DETECTION

In [ ]:
video = cv2.VideoCapture(0)

while True:
    # Grab a single frame of video
    
    ret, frame = video.read()
    # Stop the loop when we're done with the video   
    cv2.imwrite(os.path.join('video/','input.jpg'), frame)
    detections = detector.detectObjectsFromImage(input_image=os.path.join('video/','input.jpg'), output_image_path=os.path.join('video/' , "new.jpg"), minimum_percentage_probability = 20)
    print(detections)
    img_path = os.path.join('video/' , "new.jpg")
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    coords, centres = get_cent_coords(detections, img)
    yes, no = find_people(centres, coords, img)

    yes = list(set(yes))
    no = list(set(no))
    for n in no:
        if n in yes:
            yes.remove(n)

    final_img = mark_people(yes, no, coords, img)
    #cv2.imwrite('')
# Display the resulting image
    cv2.imshow('Video', final_img)

# Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Release handle to the webcam
video.release()
cv2.destroyAllWindows()